# Open file

In [3]:
import pandas
import numpy

document = pandas.read_csv("C:/Users/user/OneDrive/Desktop/MK495/final project/Data/Winter_bad.csv")
document

,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,Continent,Season
0,1,2019-1,Australia,"To all who comes to this happy place, welcome....",Disneyland_California,Oceania,Winter
1,2,2019-2,United States,Many rides were closed for maintenance. Then o...,Disneyland_California,Amercia,Winter
2,2,2019-2,United States,"Broken down rides, unhappy employees, dirty. D...",Disneyland_California,Amercia,Winter
3,2,2019-2,United States,We recently went to Balboa Island on vacation ...,Disneyland_California,Amercia,Winter
4,1,2019-2,Canada,Oh my gosh!! Where do I even begin. We drove t...,Disneyland_California,Amercia,Winter
...,...,...,...,...,...,...,...
283,1,2011-12,United States,"We love, love, love Disney World (Orlando, FL)...",Disneyland_California,Amercia,Winter
284,1,2011-12,United States,We have read that coming to Disney on Christma...,Disneyland_California,Amercia,Winter
285,1,2010-12,United States,Since the children were out of school and we w...,Disneyland_California,Amercia,Winter
286,1,2011-2,United States,Disneyland has proven to be all about getting ...,Disneyland_California,Amercia,Winter


# Transform .csv to .pkl

In [9]:
import pickle
document.to_pickle("C:/Users/user/OneDrive/Desktop/MK495/final project/Data/Winter_bad.pkl")
documents = pickle.load(open("C:/Users/user/OneDrive/Desktop/MK495/final project/Data/Winter_bad.pkl","rb"))
reviews = documents.Review_Text
print(reviews)

0      To all who comes to this happy place, welcome....
1      Many rides were closed for maintenance. Then o...
2      Broken down rides, unhappy employees, dirty. D...
3      We recently went to Balboa Island on vacation ...
4      Oh my gosh!! Where do I even begin. We drove t...
                             ...                        
283    We love, love, love Disney World (Orlando, FL)...
284    We have read that coming to Disney on Christma...
285    Since the children were out of school and we w...
286    Disneyland has proven to be all about getting ...
287    Myself, along with my two chidren ages 8 and 1...
Name: Review_Text, Length: 288, dtype: object


# No. of Reviews

In [10]:
ReviewSample = reviews
ReviewSample.shape

(288,)

# CleanedReviews

In [11]:
import spacy
nlp = spacy.load("en_core_web_sm")
CleanedReviews = ReviewSample

In [12]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer(language='english')

In [13]:
for i in range (288):
    CleanedTokens = [token.text for token in nlp(ReviewSample[i]) if not token.is_stop or token.pos_ in {'ADV','VERB'}]
    stems = [stemmer.stem(token) for token in CleanedTokens]
    CleanedReviews[i] = (" ").join(stems)
CleanedReviews

<ipython-input-13-fdbac64ec2d6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CleanedReviews[i] = (" ").join(stems)


0      come happi place , welcom . walt vision lost w...
1      ride close mainten . then ride shut wait line ...
2      broken ride , unhappi employe , dirti . defini...
3      recent went balboa island vacat famili . also ...
4      oh gosh ! ! where even begin . drove la northe...
                             ...                        
283    love , love , love disney world ( orlando , fl...
284    read come disney christma would great experi f...
285    children school work holiday decid celebr gran...
286    disneyland proven all get money not care good ...
287    , chidren age 8 17 visit disneyland california...
Name: Review_Text, Length: 288, dtype: object

# Vectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word')

vectorizer.fit(CleanedReviews)
vector = vectorizer.transform(CleanedReviews)

In [15]:
print(vector.shape)
print(vector.toarray())

(288, 3223)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# TFIDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(3,5),min_df=0.001,max_df=0.80,stop_words='english',sublinear_tf=True)
vector = tfidf_vectorizer.fit_transform(CleanedReviews)
idf = tfidf_vectorizer.idf_
tfidf = vector.sum(axis=0).A1

In [17]:
print(idf)

[5.97327951 5.97327951 5.97327951 ... 5.97327951 5.97327951 5.97327951]


In [18]:
print(tfidf)

[0.07204405 0.07204405 0.07204405 ... 0.07502736 0.07502736 0.07502736]


# Dictionary

In [19]:
SumFeature = vector.sum(axis=0)
print(SumFeature)

[[0.07204405 0.07204405 0.07204405 ... 0.07502736 0.07502736 0.07502736]]


In [20]:
Dictionary = tfidf_vectorizer.vocabulary_
findword = {value:key for key,value in Dictionary.items()}

# Silhouette_score

In [22]:
#Optimal Number of cluster
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

scores = []
for n_clusters in range(10):
    kmsclust = KMeans(n_clusters=n_clusters+2, random_state = 123)
    clusters = kmsclust.fit_predict(vector.toarray())
    
    scores.append(silhouette_score(vector.toarray(),clusters))
print(scores)

[0.00011034530031822748, 0.00012171585776167696, 0.00012318911295749522, 0.0004508674710594997, 0.0006314839223338906, 0.00011996264677768501, 0.0013082294137311673, 0.0008744037366824422, 0.001310464506286622, 0.001456468220829362]


# Find max scores

In [23]:
theset = frozenset(scores)
theset = sorted(theset, reverse=True)
thedict = {}
for j in range(10):
    positions = [i for i, x in enumerate(scores) if x == theset[j]]
    thedict[theset[j]] = positions

print('1st = ' + str(theset[0]) + ' at ' + str(thedict.get(theset[0])))

1st = 0.001456468220829362 at [9]


# KMeans

In [24]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

kms = KMeans(n_clusters = 7, random_state = 123)
clusters = kms.fit_predict(vector.toarray())
centroids = kms.cluster_centers_
print("KMclusters:", clusters)
print(" ")
print("KMcentroids:", centroids)

KMclusters: [0 0 0 0 0 0 0 3 0 0 0 1 0 0 1 0 0 0 6 0 0 0 4 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 4 0 0 1 0 0 0 0 1 0 3 0 4 0 0 4 0 1 0 0 0 4 0 2 0 0 0
 4 0 0 0 0 0 0 0 0 0 2 0 0 4 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 3 0 0 1 1 5 3 2 0 6 0 0 0 0 0 0 0 0 0 0 6 0
 0 0 0 0 0 0 0 0 4 0 0 1 0 1 1 0 0 0 0 4 3 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1 0 6 0 0 0 0 0 6 0 0 0 0 0 0 1 0 0 5 3 0 0 0 4 5 0 0 0 4 4 0 6 4 0 0 4 0
 4 6 0 6 1 5 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 6 4 0 0 0 0 0 3 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 3 0 0 1 0 0 6 1 0 0 2 0 0 4 0 0 0 0 0 0 0]
 
KMcentroids: [[ 3.32000248e-04  3.32000248e-04  3.32000248e-04 ...  3.45748214e-04
   3.45748214e-04  3.45748214e-04]
 [-5.42101086e-20 -5.42101086e-20 -5.42101086e-20 ... -5.42101086e-20
  -5.42101086e-20 -5.42101086e-20]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 5.42101086e-20  5.42101086e-20  5.42101086e-20 ...  5.42101086e-2

# Centroid population

In [25]:
from collections import Counter
count = Counter(clusters)
print(count)

Counter({0: 217, 1: 28, 4: 17, 6: 10, 3: 8, 2: 4, 5: 4})


# Assign Centroids

In [26]:
centroids.shape

(7, 64419)

In [27]:
centroids1 = centroids[0]
centroids2 = centroids[1]

# Find most word

In [28]:
theset = frozenset(centroids1)
theset = sorted(theset, reverse=True)
thedict = {}
for j in range(5):
    positions = [i for i, x in enumerate(centroids1) if x == theset[j]]
    thedict[theset[j]] = positions

print('1st = ' + str(theset[0]) + ' at ' + str(thedict.get(theset[0])))
print('2nd = ' + str(theset[1]) + ' at ' + str(thedict.get(theset[1])))
print('3rd = ' + str(theset[2]) + ' at ' + str(thedict.get(theset[2])))
print('4th = ' + str(theset[3]) + ' at ' + str(thedict.get(theset[3])))
print('5th = ' + str(theset[4]) + ' at ' + str(thedict.get(theset[4])))

1st = 0.0019301635407395594 at [61662]
2nd = 0.0016549785547554808 at [37318]
3rd = 0.001544323939726291 at [15256]
4th = 0.001348387689173169 at [20035]
5th = 0.0013137024284305629 at [47147]


In [29]:
for p in range(5):
    a = (thedict.get(theset[p]))
    b = a[0]
    print(findword[b])

wast time money
onli got ride
disneyland california adventur
fast pass ride
ride fast pass


In [30]:
theset = frozenset(centroids2)
theset = sorted(theset, reverse=True)
thedict = {}
for j in range(5):
    positions = [i for i, x in enumerate(centroids2) if x == theset[j]]
    thedict[theset[j]] = positions

print('1st = ' + str(theset[0]) + ' at ' + str(thedict.get(theset[0])))
print('2nd = ' + str(theset[1]) + ' at ' + str(thedict.get(theset[1])))
print('3rd = ' + str(theset[2]) + ' at ' + str(thedict.get(theset[2])))
print('4th = ' + str(theset[3]) + ' at ' + str(thedict.get(theset[3])))
print('5th = ' + str(theset[4]) + ' at ' + str(thedict.get(theset[4])))

1st = 0.0408643247145017 at [24050]
2nd = 0.007154844215325071 at [63658]
3rd = 0.006893301072242143 at [49844]
4th = 0.006322192387435294 at [11527, 11528, 11529, 16229, 16781, 16782, 16783, 16808, 16809, 16810, 24063, 24064, 25505, 25506, 25507, 27446, 27447, 30866, 30867, 30868, 42638, 42639, 42657, 42658, 42659, 60263, 60264, 63659, 63660]
5th = 0.005893043137578341 at [42635]


In [31]:
for p in range(5):
    a = (thedict.get(theset[p]))
    b = a[0]
    print(findword[b])

happiest place earth
worth wait hour
secur check line
crowd place earth
place earth crowd


# Sihoutte_score

In [32]:
#Optimal Number of cluster
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering

scores = []
for n_clusters in range(7):
    aggclust = AgglomerativeClustering(n_clusters=n_clusters+2)
    clusters = aggclust.fit_predict(vector.toarray())
    
    scores.append(silhouette_score(vector.toarray(),clusters))
print(scores)

[0.0038650581442779644, 0.0038592140184737483, 0.0038946883969717453, 0.003944061320296975, 0.004003886954816108, 0.004032079846386222, 0.00408015395988422]


# Find max score

In [33]:
theset = frozenset(scores)
theset = sorted(theset, reverse=True)
thedict = {}
for j in range(5):
    positions = [i for i, x in enumerate(scores) if x == theset[j]]
    thedict[theset[j]] = positions

print('1st = ' + str(theset[0]) + ' at ' + str(thedict.get(theset[0])))

1st = 0.00408015395988422 at [6]


# Agglo

In [34]:
from sklearn.cluster import AgglomerativeClustering
import numpy
aggclust = AgglomerativeClustering(n_clusters = 7)
clusters = aggclust.fit_predict(vector.toarray())
centroids = numpy.array([vector[clusters == c].mean(axis=0).A1 for c in range(6)])

In [35]:
print("Agglo_clusters:", 
      clusters)
print(" ")
print("Agglo_centroids:", 
      centroids)

Agglo_clusters: [0 0 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 6
 0 0 0 0 0 0 0 3 0 0 1 0 2 6 0 0 6 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 3 0 5 0 0 0 4 0 0 0 0 0 0 0 0 0 1 0 6 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 6 0 2 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 2
 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0
 0 0 0 0 6 0 0 2 2 0 0 0 0 0 0 0 0 1 0 6 1 0 2 0 0 0 0 0 0 0 0 0 0 6 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 5 0 0 6 0 0 0 0 0 1 0 0 0 0 0 0 0]
 
Agglo_centroids: [[0.         0.         0.         ... 0.00030499 0.00030499 0.00030499]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.

# Centroid population

In [36]:
from collections import Counter
count = Counter(clusters)
print(count)

Counter({0: 246, 6: 13, 1: 12, 2: 11, 3: 2, 5: 2, 4: 2})


# Assign Centroids

In [38]:
centroids1 = centroids[0]
centroids2 = centroids[5]

In [39]:
theset = frozenset(centroids2)
theset = sorted(theset, reverse=True)
thedict = {}
for j in range(5):
    positions = [i for i, x in enumerate(centroids2) if x == theset[j]]
    thedict[theset[j]] = positions

print('1st = ' + str(theset[0]) + ' at ' + str(thedict.get(theset[0])))
print('2nd = ' + str(theset[1]) + ' at ' + str(thedict.get(theset[1])))
print('3rd = ' + str(theset[2]) + ' at ' + str(thedict.get(theset[2])))
print('4th = ' + str(theset[3]) + ' at ' + str(thedict.get(theset[3])))
print('5th = ' + str(theset[4]) + ' at ' + str(thedict.get(theset[4])))

1st = 0.09808594318546617 at [52002, 52003]
2nd = 0.08908693396644626 at [13032]
3rd = 0.05480433218175117 at [3334, 3335, 3336, 10027, 10028, 10029, 11521, 11522, 11523, 13035, 13036, 13239, 13240, 13241, 14502, 14503, 14504, 14644, 14645, 14646, 15113, 15114, 15115, 15424, 15425, 15426, 17457, 17458, 17459, 18712, 18713, 18714, 20573, 20574, 20575, 27745, 29410, 29411, 29412, 29493, 29494, 29495, 29999, 30000, 30001, 30128, 30129, 30130, 31370, 31371, 36800, 36801, 36802, 39544, 39545, 41806, 41807, 41808, 42790, 42791, 42792, 45571, 45572, 45573, 47873, 47874, 47875, 52004, 54588, 54589, 54590, 57974, 57975, 57976, 60374, 60375, 60376, 63402, 63403, 63404]
4th = 0.050424526464682144 at [1752, 1753, 1754, 2209, 2210, 2211, 9835, 9836, 9837, 9936, 9937, 9938, 13043, 13044, 14971, 14972, 14973, 22054, 22055, 22056, 23946, 23947, 23948, 24281, 24282, 24283, 27596, 27597, 27598, 28806, 28807, 28808, 28917, 28918, 28919, 30250, 30251, 30252, 33033, 33034, 33035, 37871, 37872, 37873, 39371

In [40]:
for p in range(5):
    a = (thedict.get(theset[p]))
    b = a[0]
    print(findword[b])

spent day wait
day wait line
amus park just
80 head action
long line crowd


In [41]:
theset = frozenset(centroids1)
theset = sorted(theset, reverse=True)
thedict = {}
for j in range(5):
    positions = [i for i, x in enumerate(centroids1) if x == theset[j]]
    thedict[theset[j]] = positions

print('1st = ' + str(theset[0]) + ' at ' + str(thedict.get(theset[0])))
print('2nd = ' + str(theset[1]) + ' at ' + str(thedict.get(theset[1])))
print('3rd = ' + str(theset[2]) + ' at ' + str(thedict.get(theset[2])))
print('4th = ' + str(theset[3]) + ' at ' + str(thedict.get(theset[3])))
print('5th = ' + str(theset[4]) + ' at ' + str(thedict.get(theset[4])))

1st = 0.0019320159431115878 at [24050]
2nd = 0.0017618215766459723 at [60763]
3rd = 0.001702623936343432 at [61662]
4th = 0.001505065820276217 at [47147]
5th = 0.0015042953926954212 at [15093]


# Find most words

In [42]:
for p in range(5):
    a = (thedict.get(theset[p]))
    b = a[0]
    print(findword[b])

happiest place earth
wait time ride
wast time money
ride fast pass
disney world orlando
